# Homework Session 11 Neural networks

## Part 1

The purpose of the below is to classify days over years 2017-2018 by their corresponding mobility patterns between 10 zones in Taipei (quantified by an aggregated temporal network of subway ridership flows across the city)

In [48]:
import numpy as np
import pandas as pd
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import heapq

In [49]:
#read the data
TNet=pd.read_csv('https://raw.githubusercontent.com/CUSP2021ADS/Data/main/taipeiD_TNet2.csv',header=None);

In [50]:
TNet.head() 
#each row represents a 10x10 adjacency matrix of the normalized Taipei subway mobility network between 10 zones flattened into a 100x1 row corresponding to a single day
#days start at jan-1-2017

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.017943,0.005415,0.003590,0.008316,0.007859,0.012942,0.012196,0.019543,0.001196,0.003327,0.004588,0.016362,0.003059,0.006420,0.009864,0.005530,0.007357,0.017389,0.000923,0.001672,0.002640,0.002878,0.003133,0.001715,0.003610,0.001157,0.002406,0.004315,0.000550,0.001456,0.008631,0.007123,0.002301,0.010586,0.007468,0.010193,0.010568,0.020925,0.001346,0.002716,...,0.010876,0.007325,0.002859,0.009160,0.013417,0.009071,0.050107,0.043340,0.000679,0.003823,0.013696,0.014299,0.005237,0.015900,0.025870,0.021652,0.035190,0.049923,0.002971,0.009171,0.001081,0.001064,0.000710,0.001091,0.003131,0.008141,0.000839,0.004099,0.009125,0.005163,0.002529,0.001533,0.001860,0.002375,0.005408,0.008922,0.003945,0.011075,0.005073,0.012708
1,0.021283,0.005215,0.003530,0.009359,0.007803,0.014288,0.011185,0.019044,0.001382,0.003499,0.004859,0.016886,0.003053,0.007339,0.009820,0.005745,0.006608,0.016490,0.001023,0.001866,0.002897,0.002929,0.002973,0.001817,0.003471,0.001210,0.002197,0.004039,0.000664,0.001655,0.009672,0.007348,0.002248,0.012551,0.007475,0.011087,0.009090,0.020644,0.001560,0.003032,...,0.010139,0.006609,0.002760,0.008469,0.010956,0.009114,0.046897,0.038464,0.000647,0.003762,0.014380,0.016011,0.003765,0.017911,0.022929,0.021901,0.034270,0.040281,0.003776,0.009128,0.001298,0.001179,0.000663,0.001337,0.003490,0.008978,0.000753,0.004377,0.010360,0.005964,0.002803,0.001757,0.001783,0.002549,0.005515,0.009650,0.003596,0.009618,0.005946,0.013709
2,0.028988,0.006511,0.005591,0.012970,0.007816,0.015878,0.010973,0.015768,0.002252,0.005388,0.006879,0.013790,0.003706,0.009401,0.008878,0.006245,0.006937,0.013613,0.001545,0.002790,0.005575,0.003375,0.004373,0.003305,0.005246,0.001148,0.002829,0.003980,0.000803,0.002643,0.013623,0.008729,0.003722,0.015259,0.006972,0.012822,0.009648,0.017857,0.002672,0.004533,...,0.011613,0.007232,0.003313,0.009770,0.008924,0.009524,0.039863,0.029368,0.000581,0.005011,0.013882,0.013373,0.003620,0.017100,0.018839,0.018666,0.026413,0.030177,0.003673,0.008805,0.002030,0.001531,0.000786,0.002192,0.004388,0.010398,0.000546,0.004129,0.011692,0.009807,0.004649,0.002555,0.002672,0.004291,0.007385,0.009558,0.004293,0.008791,0.010040,0.016301
3,0.029534,0.006471,0.005615,0.013017,0.007717,0.016098,0.011182,0.015815,0.002325,0.005443,0.006955,0.014044,0.003699,0.009330,0.008967,0.006290,0.007313,0.013566,0.001511,0.002833,0.005504,0.003456,0.004210,0.003239,0.005267,0.001098,0.002910,0.003914,0.000742,0.002519,0.013751,0.008552,0.003736,0.014924,0.006757,0.012755,0.009960,0.017484,0.002665,0.004498,...,0.011870,0.007473,0.003513,0.010152,0.009209,0.009930,0.041379,0.029797,0.000618,0.005187,0.013526,0.012225,0.003561,0.016417,0.018527,0.017725,0.025343,0.030699,0.003375,0.007993,0.002014,0.001469,0.000773,0.002228,0.004599,0.010936,0.000596,0.004077,0.012252,0.009988,0.004611,0.002473,0.002636,0.004195,0.007255,0.009487,0.004316,0.008729,0.010296,0.016437
4,0.029333,0.006525,0.005727,0.013098,0.007692,0.016358,0.011000,0.015677,0.002344,0.005527,0.006860,0.013586,0.003710,0.009243,0.008994,0.006580,0.007113,0.014127,0.001536,0.002922,0.005472,0.003366,0.004166,0.003233,0.005255,0.001152,0.002889,0.003851,0.000735,0.002525,0.013695,0.008369,0.003819,0.015103,0.006924,0.012921,0.009824,0.017778,0.002605,0.004648,...,0.011968,0.007428,0.003594,0.010037,0.009058,0.009952,0.040614,0.030371,0.000633,0.005188,0.013355,0.011994,0.003674,0.016204,0.018132,0.017880,0.025106,0.030883,0.003316,0.008219,0.002130,0.001476,0.000823,0.002186,0.004413,0.010712,0.000562,0.004160,0.011789,0.009981,0.004694,0.002515,0.002677,0.004222,0.007269,0.009921,0.004387,0.008923,0.010381,0.016914


In [51]:
#convert to an array and scale the data
X=np.array(TNet);

In [52]:
X=MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(X)

In [53]:
X.shape

(669, 100)

In [54]:
#define day of the week corresponding to each day of observation; 0-Sunday, 1-Monday,...,6-Saturday
y=np.array(range(669))%7; y[:10]

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2])

In [55]:
yc=np_utils.to_categorical(y) #get categorical binary variables isSunday, isMonday,...
yc[:5]

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [71]:
X_test=X[400:,:]; X_train=X[:400,:]; #split the data into training and test
y_test=yb[400:]; y_train=yb[:400]
y_test

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0])

## Task 1. Classify weekdays/weekends
Label the rows with ones for weekends, zeros for weekdays.
Train a neural network with 4 layers of 30,10,3 and 1 (output) neurons over the training sample against this label, evaluating its performance over the test sample. Report the acheived accuracy (categorical) over the test sample

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 100 epochs, batch_size=20. 

In [69]:
# convert to ones for weekends 0 for weekdays



yb =np.array([1 if i%7 == 0 or i%7 == 6 else 0 for i in range(669)]); yb[:10]

yb[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [79]:
np.random.seed(2019)
dim = X_train.shape[1]
test = y_test


model = Sequential()
model.add(Dense(30, activation='relu', input_dim=dim))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=20, verbose=2)
print(accuracy_score(y_test, model.predict_classes(X_test)))


Epoch 1/100
20/20 - 1s - loss: 0.7062 - categorical_accuracy: 1.0000 - val_loss: 0.6536 - val_categorical_accuracy: 1.0000
Epoch 2/100
20/20 - 0s - loss: 0.6383 - categorical_accuracy: 1.0000 - val_loss: 0.6214 - val_categorical_accuracy: 1.0000
Epoch 3/100
20/20 - 0s - loss: 0.6090 - categorical_accuracy: 1.0000 - val_loss: 0.5927 - val_categorical_accuracy: 1.0000
Epoch 4/100
20/20 - 0s - loss: 0.5823 - categorical_accuracy: 1.0000 - val_loss: 0.5615 - val_categorical_accuracy: 1.0000
Epoch 5/100
20/20 - 0s - loss: 0.5533 - categorical_accuracy: 1.0000 - val_loss: 0.5460 - val_categorical_accuracy: 1.0000
Epoch 6/100
20/20 - 0s - loss: 0.5414 - categorical_accuracy: 1.0000 - val_loss: 0.5355 - val_categorical_accuracy: 1.0000
Epoch 7/100
20/20 - 0s - loss: 0.5308 - categorical_accuracy: 1.0000 - val_loss: 0.5250 - val_categorical_accuracy: 1.0000
Epoch 8/100
20/20 - 0s - loss: 0.5204 - categorical_accuracy: 1.0000 - val_loss: 0.5165 - val_categorical_accuracy: 1.0000
Epoch 9/100
20/2

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


## Task 2. Classify all days of the week
Train a neural network against the origial categorical label. Use 5 layers of 40,15,5 and 7 (outputs, representing probabilities for a current input to correspond to each of the weekdays) neurons over the training sample, evaluating its performance over the test sample (use 'categorical_accurary'). Report the acheived accuracy (categorical) over the test sample.

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 200 epochs, batch_size=20

In [83]:
y_test=yc[400:,:]; y_train=yc[:400,:]
y_test


array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [84]:
np.random.seed(2019)
dim = X_train.shape[1]

model = Sequential()
model.add(Dense(40, activation='relu', input_dim=dim))
model.add(Dense(15, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=20, verbose=2)
print(accuracy_score(y_test, model.predict_classes(X_test)))


Epoch 1/200


ValueError: ignored

## part 2

Use the same datasets as in the Advanced NN lab and further train NN with different architectures

## Task 1

Try the facial recognition task from NN lab 2 with different model specifications and report model accuracy:
1. Try different dropouts - 0.2, 0.3, 0.4
2. Try different convolution windows: (3$\times$3, 10$\times$10)

In [18]:
# load the data

from sklearn.datasets import fetch_lfw_people 
from keras.datasets import mnist

lfw_people = fetch_lfw_people(min_faces_per_person = 70, resize = 0.4) 
  
# the images arrays to find the shapes (for plotting) 
n_samples, h, w = lfw_people.images.shape 
  
# Instead of providing 2D data, X has data already in the form  of a vector that 
# is required in this approach. 
X = lfw_people.data 
n_features = X.shape[1] 

y = lfw_people.target 
target_names = lfw_people.target_names 
n_classes = target_names.shape[0] 

In [19]:
# lets split and reform the dataset in the format compatible for CNN
# basically we are restructuring the data into grid shapes which are apt for convolutions and further steps

X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size = 0.2)

X_train_pp = (X_train.reshape(X_train.shape[0], 50, 37, 1).astype('float32')) / 255 # normalize
X_test_pp = (X_test.reshape(X_test.shape[0], 50, 37, 1).astype('float32')) / 255

y_train_pp = np_utils.to_categorical(y_train)
y_test_pp = np_utils.to_categorical(y_test)

num_classes = y_train_pp.shape[1]

In [20]:

# model dropout .2

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

accuracy_score(y_test, model.predict_classes(X_test_pp))


Epoch 1/50
6/6 - 4s - loss: 1.7676 - accuracy: 0.3641 - val_loss: 1.6411 - val_accuracy: 0.4380
Epoch 2/50
6/6 - 3s - loss: 1.6728 - accuracy: 0.4049 - val_loss: 1.5984 - val_accuracy: 0.4380
Epoch 3/50
6/6 - 3s - loss: 1.6131 - accuracy: 0.4087 - val_loss: 1.5590 - val_accuracy: 0.5078
Epoch 4/50
6/6 - 3s - loss: 1.5440 - accuracy: 0.4447 - val_loss: 1.5209 - val_accuracy: 0.4651
Epoch 5/50
6/6 - 3s - loss: 1.4226 - accuracy: 0.4951 - val_loss: 1.4103 - val_accuracy: 0.5116
Epoch 6/50
6/6 - 3s - loss: 1.3346 - accuracy: 0.5340 - val_loss: 1.3610 - val_accuracy: 0.5504
Epoch 7/50
6/6 - 3s - loss: 1.2524 - accuracy: 0.5291 - val_loss: 1.2248 - val_accuracy: 0.5853
Epoch 8/50
6/6 - 3s - loss: 1.0877 - accuracy: 0.5942 - val_loss: 1.1928 - val_accuracy: 0.6047
Epoch 9/50
6/6 - 3s - loss: 1.0019 - accuracy: 0.6583 - val_loss: 1.0834 - val_accuracy: 0.6318
Epoch 10/50
6/6 - 3s - loss: 0.8511 - accuracy: 0.7388 - val_loss: 1.0250 - val_accuracy: 0.6705
Epoch 11/50
6/6 - 3s - loss: 0.8093 - a

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.8565891472868217

In [21]:

# model dropout .3

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

accuracy_score(y_test, model.predict_classes(X_test_pp))


Epoch 1/50
6/6 - 4s - loss: 1.7503 - accuracy: 0.3573 - val_loss: 1.6015 - val_accuracy: 0.4380
Epoch 2/50
6/6 - 3s - loss: 1.6589 - accuracy: 0.4049 - val_loss: 1.5661 - val_accuracy: 0.4380
Epoch 3/50
6/6 - 3s - loss: 1.6218 - accuracy: 0.4214 - val_loss: 1.5364 - val_accuracy: 0.4457
Epoch 4/50
6/6 - 3s - loss: 1.5392 - accuracy: 0.4670 - val_loss: 1.4676 - val_accuracy: 0.4845
Epoch 5/50
6/6 - 3s - loss: 1.4492 - accuracy: 0.4806 - val_loss: 1.4171 - val_accuracy: 0.5000
Epoch 6/50
6/6 - 3s - loss: 1.3699 - accuracy: 0.5000 - val_loss: 1.3482 - val_accuracy: 0.5465
Epoch 7/50
6/6 - 3s - loss: 1.2660 - accuracy: 0.5359 - val_loss: 1.2769 - val_accuracy: 0.5698
Epoch 8/50
6/6 - 3s - loss: 1.1591 - accuracy: 0.5816 - val_loss: 1.2911 - val_accuracy: 0.5155
Epoch 9/50
6/6 - 3s - loss: 1.1151 - accuracy: 0.6068 - val_loss: 1.1968 - val_accuracy: 0.5891
Epoch 10/50
6/6 - 3s - loss: 1.0634 - accuracy: 0.6330 - val_loss: 1.1032 - val_accuracy: 0.6047
Epoch 11/50
6/6 - 3s - loss: 0.9967 - a

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.8372093023255814

In [22]:

# model dropout .4

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

accuracy_score(y_test, model.predict_classes(X_test_pp))


Epoch 1/50
6/6 - 4s - loss: 1.8878 - accuracy: 0.2466 - val_loss: 1.6690 - val_accuracy: 0.4380
Epoch 2/50
6/6 - 3s - loss: 1.6925 - accuracy: 0.4049 - val_loss: 1.6290 - val_accuracy: 0.4380
Epoch 3/50
6/6 - 3s - loss: 1.6793 - accuracy: 0.4049 - val_loss: 1.6076 - val_accuracy: 0.4380
Epoch 4/50
6/6 - 3s - loss: 1.6634 - accuracy: 0.4078 - val_loss: 1.6036 - val_accuracy: 0.4380
Epoch 5/50
6/6 - 3s - loss: 1.6442 - accuracy: 0.4049 - val_loss: 1.5726 - val_accuracy: 0.4457
Epoch 6/50
6/6 - 3s - loss: 1.6028 - accuracy: 0.4320 - val_loss: 1.5216 - val_accuracy: 0.4457
Epoch 7/50
6/6 - 3s - loss: 1.5283 - accuracy: 0.4650 - val_loss: 1.4899 - val_accuracy: 0.4457
Epoch 8/50
6/6 - 3s - loss: 1.4588 - accuracy: 0.4689 - val_loss: 1.4516 - val_accuracy: 0.4651
Epoch 9/50
6/6 - 3s - loss: 1.3512 - accuracy: 0.5136 - val_loss: 1.3183 - val_accuracy: 0.5310
Epoch 10/50
6/6 - 3s - loss: 1.2703 - accuracy: 0.5398 - val_loss: 1.2365 - val_accuracy: 0.5853
Epoch 11/50
6/6 - 3s - loss: 1.1647 - a

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.813953488372093

In [23]:

# model convolution window 3x3

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

accuracy_score(y_test, model.predict_classes(X_test_pp))


Epoch 1/50
6/6 - 3s - loss: 1.9877 - accuracy: 0.2301 - val_loss: 1.7606 - val_accuracy: 0.4380
Epoch 2/50
6/6 - 2s - loss: 1.7353 - accuracy: 0.4049 - val_loss: 1.6429 - val_accuracy: 0.4380
Epoch 3/50
6/6 - 2s - loss: 1.6864 - accuracy: 0.4049 - val_loss: 1.6133 - val_accuracy: 0.4380
Epoch 4/50
6/6 - 2s - loss: 1.6697 - accuracy: 0.4049 - val_loss: 1.5975 - val_accuracy: 0.4380
Epoch 5/50
6/6 - 2s - loss: 1.6476 - accuracy: 0.4049 - val_loss: 1.5605 - val_accuracy: 0.4380
Epoch 6/50
6/6 - 2s - loss: 1.5888 - accuracy: 0.4146 - val_loss: 1.5444 - val_accuracy: 0.5233
Epoch 7/50
6/6 - 2s - loss: 1.5148 - accuracy: 0.4456 - val_loss: 1.4938 - val_accuracy: 0.5271
Epoch 8/50
6/6 - 2s - loss: 1.4199 - accuracy: 0.4825 - val_loss: 1.6316 - val_accuracy: 0.3101
Epoch 9/50
6/6 - 2s - loss: 1.4559 - accuracy: 0.4311 - val_loss: 1.4172 - val_accuracy: 0.4767
Epoch 10/50
6/6 - 2s - loss: 1.3020 - accuracy: 0.5068 - val_loss: 1.3121 - val_accuracy: 0.5349
Epoch 11/50
6/6 - 2s - loss: 1.1943 - a

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.8449612403100775

In [24]:

# model convolution window 10 x 10

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (10, 10), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (10, 10), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

accuracy_score(y_test, model.predict_classes(X_test_pp))


Epoch 1/50
6/6 - 4s - loss: 1.9162 - accuracy: 0.2621 - val_loss: 1.7830 - val_accuracy: 0.4380
Epoch 2/50
6/6 - 4s - loss: 1.7699 - accuracy: 0.4049 - val_loss: 1.6640 - val_accuracy: 0.4380
Epoch 3/50
6/6 - 4s - loss: 1.7258 - accuracy: 0.4049 - val_loss: 1.6293 - val_accuracy: 0.4380
Epoch 4/50
6/6 - 4s - loss: 1.7119 - accuracy: 0.4049 - val_loss: 1.6676 - val_accuracy: 0.4380
Epoch 5/50
6/6 - 4s - loss: 1.6993 - accuracy: 0.4049 - val_loss: 1.6318 - val_accuracy: 0.4380
Epoch 6/50
6/6 - 4s - loss: 1.6912 - accuracy: 0.4049 - val_loss: 1.6452 - val_accuracy: 0.4380
Epoch 7/50
6/6 - 4s - loss: 1.6819 - accuracy: 0.4049 - val_loss: 1.6076 - val_accuracy: 0.4380
Epoch 8/50
6/6 - 4s - loss: 1.6795 - accuracy: 0.4049 - val_loss: 1.6277 - val_accuracy: 0.4380
Epoch 9/50
6/6 - 4s - loss: 1.6647 - accuracy: 0.4049 - val_loss: 1.6011 - val_accuracy: 0.4380
Epoch 10/50
6/6 - 4s - loss: 1.6493 - accuracy: 0.4049 - val_loss: 1.5775 - val_accuracy: 0.4380
Epoch 11/50
6/6 - 4s - loss: 1.6353 - a

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.7596899224806202

## Task 2

1. Try different word count in the next-word prediction task with a)8 and b)12 previous words and report accuracy for each. Use the same Sherlock data.

2. Try different LSTM architectures and comment on model performace for each. 
    - Add a dropout layer with 0.25
    - Change number of neurons to a) 64 b) 256 neurons
    
    Train for 10 epochs 

In [26]:
path = 'sherlock.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 581889


In [27]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [28]:
# unique sorted words list
unique_words = np.unique(words)

# dictionary(<key: value>) with each word form the unique_words list as key and its corresponding position as value.
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [29]:
#With 8 previous words

WORD_LENGTH = 8
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print('sequence:', prev_words[10101])
print('next word in sequence:', next_words[10101])

sequence: ['of', 'twenty', 'one', 'years', 'are', 'eligible', 'apply', 'in']
next word in sequence: person


In [30]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [31]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [32]:

# training - just 2 epochs - should take around 5-7 mins for 2 epochs

np.random.seed(2002)
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

Epoch 1/2
811/811 [==============================] - 370s 453ms/step - loss: 6.3230 - accuracy: 0.0838 - val_loss: 7.1331 - val_accuracy: 0.1053
Epoch 2/2
811/811 [==============================] - 366s 451ms/step - loss: 5.6483 - accuracy: 0.1448 - val_loss: 7.9011 - val_accuracy: 0.1058


In [33]:
# function for preparing input as vecotrs

def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x

# function for n choose best possible words after prediction
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

# for prediction
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [86]:

# probabilities for predicted words
q =  "Your life will never be the same again"

seq = " ".join(tokenizer.tokenize(q.lower())[0:8])

prob = model.predict(prepare_input(seq), verbose=0)[0] # calculate probabilities
[prob[key] for key in np.argsort(prob)[-5:][::-1]]  # print for predicted words

your
life
will
never
be
the
same
again


ValueError: ignored

In [ ]:
#Now with 12 previous words
WORD_LENGTH = 12
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print('sequence:', prev_words[10101])
print('next word in sequence:', next_words[10101])

X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [ ]:

# training - just 2 epochs - should take around 5-7 mins for 2 epochs

np.random.seed(2002)
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

In [ ]:

# probabilities for predicted words

prob = model.predict(prepare_input(seq), verbose=0)[0] # calculate probabilities
[prob[key] for key in np.argsort(prob)[-5:][::-1]]  # print for predicted words

In [ ]:
# With dropout layer .25

model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Dropout(0.25))
model.add(Activation('softmax'))



# training - just 2 epochs - should take around 5-7 mins for 2 epochs

np.random.seed(2002)
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history


# probabilities for predicted words

prob = model.predict(prepare_input(seq), verbose=0)[0] # calculate probabilities
[prob[key] for key in np.argsort(prob)[-5:][::-1]]  # print for predicted words

In [ ]:
# With 64 neurons

model = Sequential()
model.add(LSTM(64, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))



# training - just 2 epochs - should take around 5-7 mins for 2 epochs

np.random.seed(2002)
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history


# probabilities for predicted words

prob = model.predict(prepare_input(seq), verbose=0)[0] # calculate probabilities
[prob[key] for key in np.argsort(prob)[-5:][::-1]]  # print for predicted words

In [ ]:
# With 256 neurons

model = Sequential()
model.add(LSTM(256, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Dropout(0.25))
model.add(Activation('softmax'))



# training - just 2 epochs - should take around 5-7 mins for 2 epochs

np.random.seed(2002)
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history


# probabilities for predicted words

prob = model.predict(prepare_input(seq), verbose=0)[0] # calculate probabilities
[prob[key] for key in np.argsort(prob)[-5:][::-1]]  # print for predicted words